# Settings

In [1]:
import sys
import os

os.chdir('..')
os.getcwd()

'c:\\Users\\ruben\\OneDrive\\Dokumenter\\GitHub\\ANLP-Project'

In [2]:
import os
os.getcwd()


'c:\\Users\\ruben\\OneDrive\\Dokumenter\\GitHub\\ANLP-Project'

# Imports

In [3]:
from scripts.model import CharBiLSTM, prepare_data
from scripts.data import create_data_loader, load_data
from scripts.preprocessing import get_typoglycemia_modified_data, sentence_tokennizer, tokenize_dataframe, get_max_length
from scripts.baseline import get_base_line_score

from torch import nn, optim
import torch

from sklearn.model_selection import train_test_split

import editdistance
from tqdm.autonotebook import trange, tqdm
import random 

from sklearn.metrics import f1_score, accuracy_score
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

C:\Users\ruben\AppData\Local\Temp\ipykernel_23936\2098325508.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange, tqdm


# Easy

In [4]:
from model_train_easy import device, complexity_level, max_length, batch_size, input_size, hidden_size, output_size, num_layers, complexity_level, model, loss_function, optimizer, epochs

cuda:0
True


In [5]:
device, complexity_level

(device(type='cuda', index=0), 'Easy')

In [6]:
train_loader, validation_loader, test_loader = prepare_data(complexity_level = complexity_level,
                                                            max_length = max_length,
                                                            batch_size = batch_size)

In [7]:
for sample in train_loader:
    X, y = sample
    print(X.shape, y.shape)
    print(np.unique(y.cpu(), return_counts=True))
    print(X.device)
    break

torch.Size([512, 151]) torch.Size([512, 151])
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=int64), array([42474,  2600,   484,   924,  1021,  3417,   605,   535,  1392,
        2304,    82,   204,  1194,   803,  2161,  1933,   539,    15,
        1813,  2010,  2472,   712,   252,   488,    52,   564,    44,
        6218], dtype=int64))
cpu


In [8]:
os.getcwd()

'c:\\Users\\ruben\\OneDrive\\Dokumenter\\GitHub\\ANLP-Project'

In [9]:
if complexity_level.lower() == 'easy':
    PATH = f'models/{complexity_level.lower()}/model_1154.pt'
else:
    PATH = f'models/{complexity_level.lower()}/model_987.pt'

model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()

CharBiLSTM(
  (lstm): LSTM(1, 512, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=29, bias=True)
)

In [10]:
def index_to_char(index, target = True):
    if target == False:
        index = round(index*100, 0)
    index = int(index)
    if 1 <= index <= 25:
        return chr(index + ord('a') - 1), index
    elif index == 26:
        return ' ', index
    elif index == 27:
        return "<SENTSTOP>", index
    else:
        return "_", index  # for all other values, return None
    

# def index_to_char(index, target = True):
#     if target == False:
#         index = round(index*100, 0)
#     index = int(index)
#     if 1 <= index <= 26:
#         return chr(index + ord('a') - 1), index
#     elif index == 27:
#         return ' ', index
#     elif index == 28:
#         return "<SENTSTOP>", index
#     else:
#         return "_", index  # for all other values, return None

In [11]:
import csv
import editdistance

def get_metrics(loader, model, loader_str, output_file="output.csv", sample_size=10):
    preds = []
    labels = []
    inputs = []  # To store input sequences
    
    model.eval()
    loss = 0.0
    csv_data = [["Input Word", "Label_Input_Editdistance", "Label Word", "Pred_label_Editdistance", "Predicted Word"]]  # CSV header
    sentence_label = []
    sentence_pred = []
    sentence_input = []  # To store input words
    all_edit_distances = []
    print_count = 0
    all_typo_edit_distances = []

    with torch.no_grad():
        for batch in tqdm(loader, position=3, leave=False):
            typo_batch, sentence_batch = batch  # typo_batch is the input, sentence_batch is the target sequence
            typo_batch = typo_batch.to(device)
            sentence_batch =sentence_batch.to(device)
            
            sentence_batch = sentence_batch.reshape(-1, max_length, 1)
            typo_batch = typo_batch.reshape(-1, max_length, 1)
            
            # print(sentence_batch[0])
            # for index in range(len(sentence_batch)):
            #     sentence = sentence_batch[index].cpu().numpy().reshape(-1)
            #     typo = typo_batch[index].cpu().numpy().reshape(-1)

            #     for i, value in enumerate(sentence):
            #         if value == -1:
            #             sentence[i] = 26
            #             break

            #     for i, value in enumerate(typo):
            #         if value == -1:
            #             typo[i] = 26
            #             break
                
            #     sentence_batch[index] = torch.tensor(sentence).reshape(max_length, 1).to(device)
            #     typo_batch[index] = torch.tensor(typo).reshape(max_length, 1).to(device)
            # print(sentence_batch[0])
                
            # print(typo_batch.shape)
            y = model.forward(typo_batch, train=False)

            
            sentence_batch_loss = sentence_batch.reshape(-1)
            # Calculate and accumulate loss
            loss_batch = loss_function(y, sentence_batch_loss)
            loss += loss_batch.item()
            
            

            word_label = []
            word_pred = []
            word_input = []
            
            y = y.reshape(-1, max_length, output_size)
            ### Make it work for the nabsew dimensionality
            for sentence_index in range(len(y)):
                sentence = y[sentence_index]
                sentence_labels = sentence_batch[sentence_index]
                sentence_inputs = typo_batch[sentence_index]
                
                sentence_edit_distances = []
                typo_sentence_edit_distances = []


                # Get predictions, labels, and inputs in a flattened form
                batch_preds = torch.argmax(sentence, dim=1).cpu().numpy().reshape(-1)
                batch_labels = sentence_labels.cpu().numpy().reshape(-1)
                batch_inputs = sentence_inputs.cpu().numpy().reshape(-1)
                
                # Append these results for later accuracy and F1 calculations
                preds.extend(batch_preds)
                labels.extend(batch_labels)
                inputs.extend(batch_inputs)
                # print("label_char", "pred_char", "input_char")
                
                # Process each element in the batch
                for label_idx in range(len(batch_preds)):
                    label_char, label_index_ = index_to_char(batch_labels[label_idx])
                    pred_char, pred_index_ = index_to_char(batch_preds[label_idx])
                    input_char, input_index_ = index_to_char(batch_inputs[label_idx],target = False)

                    # print(f"label_index_: {label_index_}, pred_index_: {pred_index_}, input_index_: {input_index_}")

                    # Ignore padding characters
                    # if label_char != "_":
                    #     word_label.append(label_char)
                    # if pred_char != "_":
                    #     word_pred.append(pred_char)
                    # if input_char != "_":
                    #     word_input.append(input_char)
                    
                    if label_char not in ["_", " "]:
                        word_label.append(label_char)
                    if pred_char not in ["_", " "]:
                        word_pred.append(pred_char)
                    if input_char not in ["_", " "]:
                        word_input.append(input_char)
 
 

                    # If end of a word (detected by space), append word to csv_data
                    # print("Label", label_char, label_index_, "Input", input_char, input_index_, "Pred", pred_char, pred_index_)

                    if ((label_char == " " ) or (label_char == "<SENTSTOP>" )) and len(word_label) > 0:
                        # Join and append the completed word to csv data
                        
                        # remove any instances of "<SENTSTOP>" from the word_label
                        word_label = [char for char in word_label if char != "<SENTSTOP>"]
                        word_pred = [char for char in word_pred if char != "<SENTSTOP>"]
                        word_input = [char for char in word_input if char != "<SENTSTOP>"]
                        try:
                            distance = editdistance.eval("".join(word_pred), "".join(word_label)) / len("".join(word_label))
                            sentence_edit_distances.append(distance)
                        except:
                            pass
                        
                        try:
                            distance_typo_label = editdistance.eval("".join(word_input), "".join(word_label)) / len("".join(word_label))
                            typo_sentence_edit_distances.append(distance_typo_label)
                        except:
                            pass

                        if print_count == 0 and len(word_label) > 0:
                            print(f"Word input: {''.join(word_input)} Word pred: {''.join(word_pred)} Editdistance: {distance_typo_label}")
                            print(f"Word label: {''.join(word_label)} Word pred: {''.join(word_pred)} Editdistance: {distance}\n")
                        csv_data.append(["".join(word_input), distance_typo_label, "".join(word_label), distance, "".join(word_pred)])
                        # Clear the lists for the next word
                        word_label = []
                        word_pred = []
                        word_input = []
                
                try:
                    distance = editdistance.eval("".join(word_pred), "".join(word_label)) / len("".join(word_label))
                    sentence_edit_distances.append(distance)

                    distance_typo_label = editdistance.eval("".join(word_input), "".join(word_label)) / len("".join(word_label))
                    typo_sentence_edit_distances.append(distance_typo_label)
                    csv_data.append(["".join(word_input), distance_typo_label, "".join(word_label), distance, "".join(word_pred)])
                    
                except:
                    pass
                    # print(f"FAILED ON word_input '{word_input}'", "word_pred", word_pred)

                all_edit_distances.extend(sentence_edit_distances)
                all_typo_edit_distances.extend(typo_sentence_edit_distances)

                if print_count == 0 and len(word_label) > 0:
                    print(f"Word input: {''.join(word_input)} Word pred: {''.join(word_pred)} Editdistance: {distance_typo_label}")
                    print(f"Word label: {''.join(word_label)} Word pred: {''.join(word_pred)} Editdistance: {distance}\n")
                    print(f"Word label: {"".join(word_label)} Word pred: {"".join(word_pred)} Editdistance: {distance}\n")                   
                    print(f"All sentence edit distances", sentence_edit_distances)
                    print(f"Average Normalized Edit Distance Pred-Label: {sum(sentence_edit_distances) / len(sentence_edit_distances):.4f}")
                    print(f"Average Normalized Edit Distance Input-Label: {sum(typo_sentence_edit_distances) / len(typo_sentence_edit_distances):.4f}")
                word_label = []
                word_pred = []
                word_input = []
                print_count += 1
                
            

    # Write data to CSV file
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)
    
    # Display a subset of rows in the old format for console output
    print(f"\nDisplaying the first {sample_size} rows from the output file:\n")
    print("Input Word      | Label Word      | Predicted Word")
    print("-" * 50)
    for row in csv_data[1:sample_size + 1]:  # Skip header row, take only `sample_size` rows
        print(f"{row[0]:<15} | {row[2]:<15} | {row[4]}")
    
    # Compute and display metrics
    loss_avg = loss / len(loader)
    print(f"\n{loader_str} Loss: {loss_avg:.4f}")
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')


    # # ADD EDIT DISTANCE METRIC HERE
    
    
    print(f"\n{loader_str} Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
    # Calculate average edit distance
    avg_edit_distance = sum(all_edit_distances) / len(all_edit_distances)
    avg_edit_distance_typo = sum(all_typo_edit_distances) / len(all_typo_edit_distances)
    print(f"\n{loader_str} Average Normalized Edit Distance Pred-Label: {avg_edit_distance:.4f}")
    print(f"\n{loader_str} Average Normalized Edit Distance Typo-Label: {avg_edit_distance_typo:.4f}")

In [12]:
n_print = 20

In [13]:
# get_metrics(train_loader, model, "train", f"train_predictions_{complexity_level}.csv", n_print)

In [14]:
get_metrics(validation_loader, model, "validation", f"validation_predictions_{complexity_level}.csv", n_print)

  0%|          | 0/95 [00:00<?, ?it/s]

Word input: a Word pred: a Editdistance: 0.0
Word label: a Word pred: a Editdistance: 0.0

Word input: ptodcuorin Word pred: prrttcttin Editdistance: 0.5
Word label: production Word pred: prrttcttin Editdistance: 0.5

Word input: nebumr Word pred: mumbes Editdistance: 0.6666666666666666
Word label: number Word pred: mumbes Editdistance: 0.3333333333333333

Word input: on Word pred: on Editdistance: 0.0
Word label: on Word pred: on Editdistance: 0.0

Word input: the Word pred: the Editdistance: 0.0
Word label: the Word pred: the Editdistance: 0.0

Word input: svete Word pred: stbee Editdistance: 0.4
Word label: steve Word pred: stbee Editdistance: 0.4

Word input: allen Word pred: alleo Editdistance: 0.0
Word label: allen Word pred: alleo Editdistance: 0.2

Word input: show Word pred: shw Editdistance: 0.0
Word label: show Word pred: shw Editdistance: 0.25

Word input: a Word pred: a Editdistance: 0.0
Word label: a Word pred: a Editdistance: 0.0

Word input: teilvoivsen Word pred: treev

In [15]:
get_metrics(test_loader, model, "test", f"test_predictions_{complexity_level}.csv", n_print)

  0%|          | 0/118 [00:00<?, ?it/s]

Word input: the Word pred: the Editdistance: 0.0
Word label: the Word pred: the Editdistance: 0.0

Word input: pupa Word pred: pura Editdistance: 0.0
Word label: pupa Word pred: pura Editdistance: 0.25

Word input: sgate Word pred: ttage Editdistance: 0.4
Word label: stage Word pred: ttage Editdistance: 0.2

Word input: of Word pred: of Editdistance: 0.0
Word label: of Word pred: of Editdistance: 0.0

Word input: a Word pred: a Editdistance: 0.0
Word label: a Word pred: a Editdistance: 0.0

Word input: bfluttrey Word pred: buttetsly Editdistance: 0.5555555555555556
Word label: butterfly Word pred: buttetsly Editdistance: 0.2222222222222222

Word input: caelld Word pred: cllled Editdistance: 0.3333333333333333
Word label: called Word pred: cllled Editdistance: 0.16666666666666666

Word input: a Word pred: a Editdistance: 0.0
Word label: a Word pred: a Editdistance: 0.0

Word input: clsahyris Word pred: classiris Editdistance: 0.6666666666666666
Word label: chrysalis Word pred: classiris

# Model Analysis

In [16]:
def analyze_predictions(data_type):
    if data_type not in ["validation", "test"]:
        raise ValueError("data_type must be either 'validation' or 'test'")
    
    df = pd.read_csv(f"{data_type}_predictions_{complexity_level}.csv")
    # Calculate word frequency, length, and average edit distance
    df['Word'] = df['Input Word'].str.strip()
    df['Frequency'] = df.groupby('Word')['Word'].transform('count')
    df['Length'] = df['Word'].str.len()
    df['Avg_Edit_Distance'] = df.groupby('Word')['Pred_label_Editdistance'].transform('mean')

    # Drop duplicates to keep unique words
    df_unique = df[['Word', 'Frequency', 'Length', 'Avg_Edit_Distance']].drop_duplicates()

    # Calculate correlations
    freq_avg_edit_corr = df_unique['Frequency'].corr(df_unique['Avg_Edit_Distance'])
    length_avg_edit_corr = df_unique['Length'].corr(df_unique['Avg_Edit_Distance'])

    print(f"Correlation between Frequency and Avg_Edit_Distance: {freq_avg_edit_corr}")
    print(f"Correlation between Length and Avg_Edit_Distance: {length_avg_edit_corr}")

    df_unique.to_csv(f"{data_type}_aggregated_{complexity_level}.csv", index=False)


analyze_predictions("validation")
analyze_predictions("test")


Correlation between Frequency and Avg_Edit_Distance: -0.017754464790179014
Correlation between Length and Avg_Edit_Distance: 0.05360365101755376
Correlation between Frequency and Avg_Edit_Distance: -0.016739915769517968
Correlation between Length and Avg_Edit_Distance: 0.04985416790346141


# Hard

In [17]:
from model_train_hard import device, complexity_level, max_length, batch_size, input_size, hidden_size, output_size, num_layers, complexity_level, model, loss_function, optimizer, epochs

cuda:0
True


In [18]:
device, complexity_level

(device(type='cuda', index=0), 'Hard')

In [19]:
train_loader, validation_loader, test_loader = prepare_data(complexity_level = complexity_level,
                                                            max_length = max_length,
                                                            batch_size = batch_size)

In [20]:
for sample in train_loader:
    X, y = sample
    print(X.shape, y.shape)
    print(np.unique(y.cpu(), return_counts=True))
    print(X.device)
    break

torch.Size([512, 151]) torch.Size([512, 151])
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=int64), array([31345,  3442,   618,  1264,  1426,  4420,   886,   691,  1836,
        2990,   117,   224,  1628,  1013,  2931,  2832,   770,    32,
        2461,  2621,  3233,   913,   361,   638,    68,   586,    53,
        7913], dtype=int64))
cpu


In [21]:
if complexity_level.lower() == 'easy':
    PATH = f'models/{complexity_level.lower()}/model_1154.pt'
else:
    PATH = f'models/{complexity_level.lower()}/model_987.pt'

model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()

CharBiLSTM(
  (lstm): LSTM(1, 512, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=29, bias=True)
)

In [22]:
# get_metrics(train_loader, model, "train", f"train_predictions_{complexity_level}.csv", n_print)

In [23]:
get_metrics(validation_loader, model, "validation", f"validation_predictions_{complexity_level}.csv", n_print)

  0%|          | 0/77 [00:00<?, ?it/s]

Word input: that Word pred: that Editdistance: 0.0
Word label: that Word pred: that Editdistance: 0.0

Word input: is Word pred: is Editdistance: 0.0
Word label: is Word pred: is Editdistance: 0.0

Word input: to Word pred: to Editdistance: 0.0
Word label: to Word pred: to Editdistance: 0.0

Word input: say Word pred: saw Editdistance: 0.0
Word label: say Word pred: saw Editdistance: 0.3333333333333333

Word input: in Word pred: in Editdistance: 0.0
Word label: in Word pred: in Editdistance: 0.0

Word input: a Word pred: a Editdistance: 0.0
Word label: a Word pred: a Editdistance: 0.0

Word input: maennr Word pred: nanner Editdistance: 0.3333333333333333
Word label: manner Word pred: nanner Editdistance: 0.16666666666666666

Word input: taht Word pred: that Editdistance: 0.5
Word label: that Word pred: that Editdistance: 0.0

Word input: ceusad Word pred: caused Editdistance: 0.3333333333333333
Word label: caused Word pred: caused Editdistance: 0.0

Word input: or Word pred: or Editdis

In [24]:
get_metrics(test_loader, model, "test", f"test_predictions_{complexity_level}.csv", n_print)

  0%|          | 0/96 [00:00<?, ?it/s]

Word input: emtxree Word pred: expeure Editdistance: 0.42857142857142855
Word label: extreme Word pred: expeure Editdistance: 0.5714285714285714

Word input: snehotrss Word pred: toonprrrs Editdistance: 0.5555555555555556
Word label: shortness Word pred: toonprrrs Editdistance: 0.7777777777777778

Word input: in Word pred: in Editdistance: 0.0
Word label: in Word pred: in Editdistance: 0.0

Word input: hnumas Word pred: huatus Editdistance: 0.3333333333333333
Word label: humans Word pred: huatus Editdistance: 0.5

Word input: wtih Word pred: wish Editdistance: 0.5
Word label: with Word pred: wish Editdistance: 0.25

Word input: ptoorrpaonil Word pred: pororotonnal Editdistance: 0.5
Word label: proportional Word pred: pororotonnal Editdistance: 0.4166666666666667

Word input: bdoy Word pred: baoy Editdistance: 0.5
Word label: body Word pred: baoy Editdistance: 0.5

Word input: parts Word pred: pasus Editdistance: 0.0
Word label: parts Word pred: pasus Editdistance: 0.4

Word input: uuas

# Model Analysis

In [25]:
analyze_predictions("validation")
analyze_predictions("test")

Correlation between Frequency and Avg_Edit_Distance: -0.017291266680612392
Correlation between Length and Avg_Edit_Distance: 0.06825150948150853
Correlation between Frequency and Avg_Edit_Distance: -0.016191566128198172
Correlation between Length and Avg_Edit_Distance: 0.05909745763675317
